In [80]:
import matplotlib.pyplot as plt
import csv
from textblob import TextBlob
import pandas as pd
import sklearn
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import learning_curve
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV

**Importing train, validation and test set**

In [75]:
train_data = pd.read_csv("/train (1).csv")
validation_data=pd.read_csv("/content/test.csv")
test_data=pd.read_csv("/content/validation.csv")

**Assigning labels, vectorizing the text data, applying TF-IDF**

In [97]:
# Assign features and labels
X_train = train_data['processed_text']
y_train = train_data['spam']
X_val = validation_data['processed_text']
y_val = validation_data['spam']

X_test = test_data['processed_text']
y_test = test_data['spam']

X_train = X_train.fillna('')

# Vectorize the text data using CountVectorizer
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(X_train)
X_val_counts = vectorizer.transform(X_val)
X_test_counts = vectorizer.transform(X_test)

# Apply TF-IDF transformation
tfidf_transformer = TfidfTransformer()
X_train_vec = tfidf_transformer.fit_transform(X_train_counts)
X_val_vec = tfidf_transformer.transform(X_val_counts)
X_test_vec = tfidf_transformer.transform(X_test_counts)




**Random Forest**

In [99]:
from sklearn.metrics import confusion_matrix

# Random Forest
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train_vec, y_train)
rf_predictions = rf_classifier.predict(X_test_vec)

rf_accuracy = accuracy_score(y_test, rf_predictions)
rf_precision = precision_score(y_test, rf_predictions)
rf_recall = recall_score(y_test, rf_predictions)

print("Random Forest Validation Metrics:")
print("Accuracy:", rf_accuracy)
print("Precision:", rf_precision)
print("Recall:", rf_recall)

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, rf_predictions)
print("\nConfusion Matrix:")
print(conf_matrix)


Random Forest Validation Metrics:
Accuracy: 0.9634369287020109
Precision: 0.9831081081081081
Recall: 0.9509803921568627

Confusion Matrix:
[[236   5]
 [ 15 291]]


**Hyperparameter Tuning**

In [100]:
rf_param_grid = {'n_estimators': [100, 200, 300],
                 'max_depth': [None, 10, 20]}

rf_grid_search = GridSearchCV(RandomForestClassifier(), rf_param_grid, cv=5, n_jobs=-1)
rf_grid_search.fit(X_val_vec, y_val)

print("\nRandom Forest Best Parameters:", rf_grid_search.best_params_)
print("Random Forest Best Score:", rf_grid_search.best_score_)


Random Forest Best Parameters: {'max_depth': None, 'n_estimators': 300}
Random Forest Best Score: 0.9543786488740619


**Fitting the best hyperparameters values and checking the performance of the model**

In [112]:
best_rf = RandomForestClassifier(n_estimators=rf_grid_search.best_params_['n_estimators'],
                                  max_depth=rf_grid_search.best_params_['max_depth'])

# Fit models to training data
best_rf.fit(X_train_vec, y_train)

# Predict on test set
rf_test_predictions = best_rf.predict(X_test_vec)
# Evaluate performance on test set
rf_test_accuracy = accuracy_score(y_test, rf_test_predictions)

rf_test_precision = precision_score(y_test, rf_test_predictions)
rf_test_recall = recall_score(y_test, rf_test_predictions)

print("Random Forest Validation Metrics:")
print("Accuracy:", rf_test_accuracy)
print("Precision:", rf_test_precision)
print("Recall:", rf_test_recall)

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, rf_test_predictions)
print("\nConfusion Matrix:")
print(conf_matrix)


Random Forest Validation Metrics:
Accuracy: 0.9725776965265083
Precision: 0.9932203389830508
Recall: 0.9575163398692811

Confusion Matrix:
[[239   2]
 [ 13 293]]


**Logistic Regression**

In [109]:
# Logistic Regression
lr_classifier = LogisticRegression()
lr_classifier.fit(X_train_vec, y_train)
lr_predictions = lr_classifier.predict(X_test_vec)

lr_accuracy = accuracy_score(y_test, lr_predictions)
lr_precision = precision_score(y_test, lr_predictions)
lr_recall = recall_score(y_test, lr_predictions)

print("\nLogistic Regression Validation Metrics:")
print("Accuracy:", lr_accuracy)
print("Precision:", lr_precision)
print("Recall:", lr_recall)
# Confusion Matrix
conf_matrix = confusion_matrix(y_test, lr_predictions)
print("\nConfusion Matrix:")
print(conf_matrix)



Logistic Regression Validation Metrics:
Accuracy: 0.9872029250457038
Precision: 0.9807073954983923
Recall: 0.9967320261437909

Confusion Matrix:
[[235   6]
 [  1 305]]


**Hyperparameter Tuning**

In [114]:
# Logistic Regression
lr_param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}

lr_grid_search = GridSearchCV(LogisticRegression(), lr_param_grid, cv=5, n_jobs=-1)
lr_grid_search.fit(X_val_vec, y_val)

print("\nLogistic Regression Best Parameters:", lr_grid_search.best_params_)
print("Logistic Regression Best Score:", lr_grid_search.best_score_)


Logistic Regression Best Parameters: {'C': 100}
Logistic Regression Best Score: 0.9762468723936614


**Fitting the best hyperparameters values and checking the performance of the model**

In [118]:
best_lr = LogisticRegression(C=lr_grid_search.best_params_['C'])

best_lr.fit(X_train_vec, y_train)
lr_test_predictions = best_lr.predict(X_test_vec)

lr_test_accuracy = accuracy_score(y_test, lr_test_predictions)
lr_test_precision = precision_score(y_test, lr_test_predictions)
lr_test_recall = recall_score(y_test, lr_test_predictions)

print("\nLogistic Regression Validation Metrics:")
print("Accuracy:", lr_test_accuracy)
print("Precision:", lr_test_precision)
print("Recall:", lr_test_recall)


# Confusion Matrix
conf_matrix = confusion_matrix(y_test, lr_test_predictions)
print("\nConfusion Matrix:")
print(conf_matrix)


Logistic Regression Validation Metrics:
Accuracy: 0.9908592321755028
Precision: 0.9870550161812298
Recall: 0.9967320261437909

Confusion Matrix:
[[237   4]
 [  1 305]]


**Support Vector Machine**

In [120]:
# SVM
svm_classifier = SVC()
svm_classifier.fit(X_train_vec, y_train)
svm_predictions = svm_classifier.predict(X_test_vec)

svm_accuracy = accuracy_score(y_test, svm_predictions)
svm_precision = precision_score(y_test, svm_predictions)
svm_recall = recall_score(y_test, svm_predictions)

print("\nSVM Validation Metrics:")
print("Accuracy:", svm_accuracy)
print("Precision:", svm_precision)
print("Recall:", svm_recall)
# Confusion Matrix
conf_matrix = confusion_matrix(y_test, svm_predictions)
print("\nConfusion Matrix:")
print(conf_matrix)


SVM Validation Metrics:
Accuracy: 0.9890310786106032
Precision: 0.9838709677419355
Recall: 0.9967320261437909

Confusion Matrix:
[[236   5]
 [  1 305]]


**Hyperparameter Tuning**

In [121]:
# SVM
svm_param_grid = {'C': [0.1, 1, 5],
                  'kernel': ['linear', 'rbf']}

svm_grid_search = GridSearchCV(SVC(), svm_param_grid, cv=5, n_jobs=-1)
svm_grid_search.fit(X_val_vec, y_val)

print("\nSVM Best Parameters:", svm_grid_search.best_params_)
print("SVM Best Score:", svm_grid_search.best_score_)


SVM Best Parameters: {'C': 5, 'kernel': 'linear'}
SVM Best Score: 0.9798999165971644


**Fitting the best hyperparameters values and checking the performance of the model**

In [123]:
best_svm = SVC(C=svm_grid_search.best_params_['C'],
               kernel=svm_grid_search.best_params_['kernel'])
best_svm.fit(X_train_vec, y_train)
svm_test_predictions = best_svm.predict(X_test_vec)
svm_test_accuracy = accuracy_score(y_test, svm_test_predictions)
svm_test_precision = precision_score(y_test, svm_test_predictions)
svm_test_recall = recall_score(y_test, svm_test_predictions)

print("\nSVM Validation Metrics:")
print("Accuracy:", svm_test_accuracy)
print("Precision:", svm_test_precision)
print("Recall:", svm_test_recall)
# Confusion Matrix
conf_matrix = confusion_matrix(y_test, svm_test_predictions)
print("\nConfusion Matrix:")
print(conf_matrix)


SVM Validation Metrics:
Accuracy: 0.9926873857404022
Precision: 0.9902597402597403
Recall: 0.9967320261437909

Confusion Matrix:
[[238   3]
 [  1 305]]


**From the Above 3 models, the performance of Support Vector Machine is better than Random Forest and Logistic Regression**